In [74]:
# coding: utf-8

import json

import urllib2

import numpy as np
import pandas as pd

# coding: utf-8

from collections import defaultdict
from datetime import datetime
import time
import json

import gensim
import pymongo

import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
import pymorphy2

In [69]:
from pymongo import MongoClient, DESCENDING

In [79]:
client = MongoClient()
c = client.dataframe

embs = db.embeddings
s_embeddings = db.short_embeddings
vkfull = db.vk_full

db_users_and_features = client['users_and_features']
posts = db_users_and_features.posts

In [12]:
import os

os.getcwd()

'/root/in_relaiton'

In [13]:
%%time
model = gensim.models.KeyedVectors.load_word2vec_format('web_0_300_20.bin.gz', binary=True)

CPU times: user 14.5 s, sys: 356 ms, total: 14.8 s
Wall time: 14.8 s


In [14]:
%%time
w2v = dict(zip(model.index2word, model.syn0))

CPU times: user 128 ms, sys: 32 ms, total: 160 ms
Wall time: 159 ms


In [15]:
morph = pymorphy2.MorphAnalyzer()

In [16]:
mapping = {
    'ADJF': '_ADJ',
    'NOUN': '_NOUN',
    'INFN': '_VERB',
}

def prepare_word(w, mapping, word2vec_model):
    w = morph.parse(w)[0]
    if w.tag.POS in mapping:
        new_w = '{}{}'.decode('utf-8').format(w.normal_form, mapping[w.tag.POS])
        v = vector(new_w, word2vec_model)  
        return v, new_w
    return (None, None)

def w2v_transformation(data, word2vec_model):
    """
    transform the data - it can be pandas format or just array
    :param data: data - pandas, numpy, list
    :param word2vec_model
    :return: transformed_data
    """
    dim = len(word2vec_model.itervalues().next())

    train_vectors = []
    for i, s in enumerate(data):
        words = s.split(' ')

        words_vector = None
        words_count = 0

        new_s = []
        sentence_vector = []
        for w in words:
            v, new_w = prepare_word(w, mapping, word2vec_model)
            if new_w:
                new_s.append(new_w)
                
                if isinstance(v, np.ndarray):
                    words_count += 1

                    if isinstance(words_vector, np.ndarray):
                        words_vector = words_vector + v
                    else:
                        words_vector = v

        if isinstance(words_vector, np.ndarray) and words_count > 0:
            words_vector = words_vector / words_count
        else:
            words_vector = None

        if isinstance(words_vector, np.ndarray):
            train_vectors.append(words_vector)
            
#         tfidf = TfidfVectorizer(analyzer=lambda x: x)
#         tfidf.fit(new_s)
            
#         max_idf = max(tfidf.idf_)
#         word2weight = defaultdict(
#             lambda: max_idf,
#             [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
#         )
            
#         train_w2v_tfidf.append(
#             np.array(
#                 np.mean(
#                     [word2vec_model[w] * word2weight[w] for w in new_s if w in word2vec_model]
#                     or [np.zeros(dim)],
#                     axis=0
#                 )
#             )
#         )

#     train_vectors = np.array(train_vectors)
    return train_vectors
#     return np.array(train_w2v_tfidf)

In [17]:
def vector(q, word2vec_model):
    """
    transform a single word to word2vec model - 500 digits
    :param q: just a word from w2v vocabulary
    :param word2vec_model: word2vec model
    :return: vector from 500 digits, numpy
    """
    qf = q

    if q not in word2vec_model:
        candidates_set = set()

        candidates_set.add(q.upper())
        candidates_set.add(q.lower())
        candidates_set.add(q.capitalize())

        no_results = True
        for candidate in candidates_set:
            if candidate in word2vec_model:
                qf = candidate
                no_results = False
                break

        if no_results:
            # obvious that not all elements
            # in corpus will from our vocabulary
            return None

    raw_vector = word2vec_model[qf]
    return raw_vector

In [18]:
!echo -en $(curl http://138.201.80.213:5005/get_user_info?uid=25553&fields=interests)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   452  100   452    0     0    832      0 --:--:-- --:--:-- --:--:--   830
-en {"interests": ["\u042e\u043c\u043e\u0440", "\u041d\u043e\u0432\u043e\u0441\u0442\u0438", "\u041b\u0438\u0442\u0435\u0440\u0430\u0442\u0443\u0440\u0430", "\u041f\u0443\u0442\u0435\u0448\u0435\u0441\u0442\u0432\u0438\u044f", "\u041c\u0443\u0437\u044b\u043a\u0430"], "mbti": {"probs": {"intro_extra": 0.2446371465921402, "sensor_intuit": 0.25243809819221497, "logic_ethic": 0.049992892891168594, "irratio_ratio": 0.41522619128227234}, "psy_type": "ENFP"}}


In [19]:
interests = """
Интересы
Коты
SMM
Diy
Дети - мамы
Кулинария
Футбол
Психология
Geek
Гороскоп
Програмирование
Двач
Юмор
Православие
Концерты
Рок
Знакомства
Instagram
Егэ
Дизайн
Мемы
Будизм
Философия
Карьера
Иностранные языки
Gamer
Наука
Бизнес
Эстетика
Литература
Музыка
Фриланс
Web дизайн
Саморазвитие
Недвижимость
Татуировки
Фотография
Хоккей
Выставки
Анимэ
Рэп
Поиск работы
Любовь
Зож
Бесплатно, конкурсы
Собаки
Порно
Патриоты
Dance music
Автомобили
Путешествия
Интерьеры
Медицина
Data Science
Правильное питание
Уют
Стартап
Новости
Реклама и маркетинг
Hipster
Математика
Политика
Национализм
Технологии
Мусульманин
Медитации
Искусство
Архитектура
Секс
Эзотерика
Фитнес
Цитаты
"""

mapping_interests = {}
for i, e in enumerate(interests.split('\n')):
    if e:
        mapping_interests[e.strip()] = i - 1

In [20]:
len(mapping_interests), max(mapping_interests.values()), min(mapping_interests.values())

(72, 71, 0)

In [21]:
def get_datafuel(uid):
    
    try:
        raw = urllib2.urlopen(
            'http://138.201.80.213:5005/get_user_info?uid={}&fields=interests'.format(uid)
        )
        a = np.zeros(len(mapping_interests))
        interests = json.load(raw)['interests']
        for e in interests:
            e = e.encode('utf-8')
            a[mapping_interests[e]] = 1
        return a, interests
    except Exception as e:
        return np.zeros(len(mapping_interests)), []

In [22]:
get_datafuel(34564079)

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.]), [])

In [23]:
o = json.loads("""{
    "friends": {
        "count": 0,
        "items": []
    },
    
    "basic": {
        "id": 12532338,
        "first_name": "Artash",
        "last_name": "Muradyan",
        "sex": 2,
        "country": {
            "id": 1,
            "title": "Russia"
        },
        "photo": "https://vk.com/images/camera_50.png",
        "university": 0,
        "university_name": "",
        "faculty": 0,
        "faculty_name": "",
        "graduation": 0,
        "home_town": "",
        "relation": 0,
        "about": "ехал грека в уругвай видит он там что то"
    },
    "wall": {
        "count": 0,
        "items": []
    },
    "id": 12532338,
    "groups": null
}""")

In [80]:
name_mapping = {}
interest_mapping = {}
name_counter = 0


def get_main_ohe(basics, use_interests=True):
    v = []
    
    v.append(basics['id'])

    interests = None
    if use_interests:
        emb, interests = get_datafuel(basics['id'])
        v += list(emb)
    
#     if basics['first_name'] in mapping:
#         v.append(mapping[basics['first_name']])
#     else:
#         name_counter += 1
#         mapping[basics['first_name']] = name_counter
#         v.append(name_counter)
        
    v.append(basics['sex'])
    
    city = basics['city']['id'] if 'city' in basics else -1
    v.append(city)

    country = basics['country']['id'] if 'country' in basics else -1
    v.append(country)
    
    photo = 1 if 'photo' in basics else 0
    v.append(photo)
    
    try:
        birthdate = (time.mktime(datetime.strptime(basics['bdate'], '%d.%m.%Y').timetuple())
                     if 'bdate' in basics else 788907600.0)
    except: 
        birthdate = 788907600.0
    v.append(birthdate)
    relation = basics['relation'] if 'relation' in basics else -1
    v.append(relation)
    
    university = basics['university'] if 'university' in basics else -1
    
    if 'about' in basics:
        transf = w2v_transformation([basics['about']], w2v)
        if transf is not None and len(transf) > 0:
            about = transf[0]
        else:
            about = np.zeros(300)
    else:
        about = np.zeros(300)
                
    v += list(about)
    return v, interests, birthdate

In [67]:
get_main_ohe(o['basic'])

([12532338,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  2,
  -1,
  1,
  1,
  788907600.0,
  0,
  -0.042778723,
  -0.03124951,
  -0.046841875,
  0.083490834,
  -0.045750491,
  0.047526635,
  0.0043142736,
  -0.11746912,
  -0.017969202,
  0.045542754,
  0.0054357843,
  -0.054534398,
  -0.013135087,
  -0.044075444,
  0.048176371,
  -0.0066767819,
  0.079794891,
  0.08411856,
  0.044240609,
  0.067758858,
  0.013826977,
  0.061151184,
  0.00026977994,
  -0.0015451983,
  0.03163369,
  0.029718235,
  0.011587636,
  0.029560436,
  0.017743

In [26]:
def data_dumper(rd, fname):
    with open(fname, 'a') as rf:
        json.dump(rd, rf)

In [59]:
passed_files = []

In [62]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

def get_basic():

    user_vectors = []
    count = 0
    
    count_unparsed = 0
    errors = []

    for d, dirs, files in os.walk('/var/vk-data'):
        for f in files:
            jf = None
            fname = os.path.join(d, f)
            
            if fname in passed_files:
                continue
            
            passed_files.append(fname)
            with open(fname, 'r') as rf:
                try:
#                     print 'prejson'
                    jf = json.loads(rf.read())
#                     print 'postjson'
                except:
                    continue
                
                try:
                    if jf:
                        #пользователь забанен\удалился
                        if "deactivated" in jf['basic']:
                            continue
                        
                        # у пользователя нет камерки
                        if "camera" in jf['basic']["photo_max"]:
                            continue
                    
#                     print 'preohe'

                    if 'basic' not in jf:
                        continue
                    v, interests = get_main_ohe(jf['basic'])
#                     print 'postohe'
                    
                    jf['basic']['interests'] = interests
                    jf['basic']['embedding'] = v
                
                    vkfull.insert_one(
                        {str(jf['basic']['id']): json.dumps(jf, cls=MyEncoder)}
                    )
                    
                    embs.insert_one(
                        {str(jf['basic']['id']): [str(e) for e in v]}
                    )
#                         json.dumps(v, cls=MyEncoder)
#                     )
                    
                    user_vectors.append(v)
#                 
                    if len(user_vectors) % 500 == 0:
                        data_dumper(user_vectors, '/root/in_relaiton/vectors{}.json'.format(len(user_vectors)))
                    
                    count += 1
                    print count
                    
                except Exception as e:
                    print e
                    errors.append((e, f))
                    count_unparsed += 1

#             with open(os.path.join(d, f), 'w') as rf:
#                 try:
#                     if jf:
# #                     print [type(e) for e in jf['basic']['embedding']]
#                         json.dump(jf, rf, cls=MyEncoder)
#                 except Exception as e:
#                     print type(jf['basic']['embedding']), e

    print 'unparsed: {}'.format(count_unparsed)
    try:
        data_dumper(user_vectors, '/root/in_relaiton/vectors.json')
    except:
        print 'not already saved'
    return user_vectors

In [72]:
# %%time
# user_vectors = get_basic()

In [64]:
# embs.drop()
# vkfull.drop()

In [204]:
def update_users_and_features(version=1):
    
    """
    vector - out embedding
    score_is_active - xgboost_model
    
    """
    
    for d, dirs, files in os.walk('/var/vk-data-1000'):
        for f in files:
            with open(os.path.join(d, f), 'r') as rf:
                
                local_count = 0
                lower, maximum = map(int, f.replace('.json', '').split('-')[1:])
                print lower, maximum
                ffeatures = posts.find({u'uid': {'$gt': lower, '$lt': maximum}, 
                                     'face_features': {'$exists': True}})
                count_ = ffeatures.count()
                
                print count_
                if count_ == 0:
                    print f
                    continue
                
                try:
                    json_objects = json.load(rf)
                except Exception as e:
                    print e
                
                ids_update = map(int, [e['uid'] for e in ffeatures])
                    
                
                for i, profile_info in enumerate(json_objects):
                    if profile_info:
                        
                        if 'id' in profile_info and int(profile_info['id']) not in ids_update:
                            continue
                            
                        #пользователь забанен\удалился
                        if 'deactivated' in profile_info:
                            continue
                            
                        if 'photo_max' not in profile_info:
                            continue
                        
                        # у пользователя нет камерки
                        if 'camera' in profile_info['photo_max']:
                            continue
                            
#                     print int(profile_info['id'])
                    users = posts.find_one({u'uid': int(profile_info['id']), 'face_features': {'$exists': True}}) 
                    if not users:
                        continue                    
                    
                    
                    if local_count >= count_:
                        break
                    
                    print 'feature_faces'
                    
                    v, interests, birthdate = get_main_ohe(profile_info, use_interests=False)
                    v = [float(e) for e in v]
#                     s_embeddings.insert_one(
#                         {str(profile_info['id']): [str(e) for e in v]}
#                     )
                    
                                       

#                         post_data = {
#                             u'uid': profile_info['id'],
#                             'city': profile_info['city'] if 'city' in profile_info else -1,
#                             'age': birthdate,
#                             'vk_url': 'https://vk.com/id{}'.format(profile_info['id']),
#                             'profile_embedding': v,
#                             'active_search_score': 1.,
#                             'photo_url': profile_info['photo_max'],
#                             'version': version
#                         }
#                         result = posts.insert_one(post_data)
                        
#                     elif (('active_search_score' not in users) or ('profile_embedding' not in users)):
                    posts.update(
                        { u'_id': users[u'_id'] },
                        { '$set': {
                            'city':  profile_info['city'] if 'city' in profile_info else -1,
                            'age': birthdate,
                            'vk_url': 'https://vk.com/id{}'.format(profile_info['id']),
                            'profile_embedding': v,
                            'active_search_score': 1.,
                            'photo_url': profile_info['photo_max'],
                            'version': version
                            
                        } }
                    )
    
#                     print posts.find_one({ u'_id': users[u'_id'] })
    
#                     print 'saved'
                    local_count += 1
                    print local_count
    
#                     users['city'] = profile_info['city'] if 'city' in profile_info else -1
#                     users['age'] = birthdate
#                     users['vk_url'] = 'https://vk.com/id{}'.format(profile_info['id'])
#                     users['profile_embedding'] = v
#                     users['active_search_score'] = 1.
#                     users['photo_url'] = profile_info['photo_max']
#                     users['version'] = version
#                     posts.save(users)
                                        

In [205]:
# update_users_and_features()

In [206]:
update_users_and_features(version=1)

96692000 96693000
0
profile-96692000-96693000.json
9948000 9949000
0
profile-9948000-9949000.json
53357000 53358000
0
profile-53357000-53358000.json
3753000 3754000
0
profile-3753000-3754000.json
64545000 64546000
1
feature_faces
1
20278000 20279000
1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:91: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


feature_faces
1
7714000 7715000
0
profile-7714000-7715000.json
73926000 73927000
2
feature_faces
1
feature_faces
2
50273000 50274000
0
profile-50273000-50274000.json
26184000 26185000
1
feature_faces
1
29304000 29305000
2
feature_faces
1
feature_faces
2
89214000 89215000
0
profile-89214000-89215000.json
41264000 41265000
0
profile-41264000-41265000.json
55036000 55037000
0
profile-55036000-55037000.json
13423000 13424000
0
profile-13423000-13424000.json
56233000 56234000
0
profile-56233000-56234000.json
68993000 68994000
1
feature_faces
1
69450000 69451000
0
profile-69450000-69451000.json
28811000 28812000
1
feature_faces
1
37548000 37549000
0
profile-37548000-37549000.json
43281000 43282000
0
profile-43281000-43282000.json
14894000 14895000
0
profile-14894000-14895000.json
25911000 25912000
0
profile-25911000-25912000.json
35162000 35163000
0
profile-35162000-35163000.json
21595000 21596000
0
profile-21595000-21596000.json
98029000 98030000
0
profile-98029000-98030000.json
95595000 95

0
profile-64016000-64017000.json
40933000 40934000
0
profile-40933000-40934000.json
60566000 60567000
0
profile-60566000-60567000.json
49071000 49072000
0
profile-49071000-49072000.json
2358000 2359000
0
profile-2358000-2359000.json
5051000 5052000
1
feature_faces
1
14820000 14821000
1
feature_faces
1
70038000 70039000
0
profile-70038000-70039000.json
96136000 96137000
0
profile-96136000-96137000.json
16023000 16024000
0
profile-16023000-16024000.json
24829000 24830000
0
profile-24829000-24830000.json
21589000 21590000
0
profile-21589000-21590000.json
17018000 17019000
1
feature_faces
1
13710000 13711000
2
feature_faces
1
feature_faces
2
9709000 9710000
0
profile-9709000-9710000.json
24220000 24221000
0
profile-24220000-24221000.json
73657000 73658000
0
profile-73657000-73658000.json
90463000 90464000
0
profile-90463000-90464000.json
31512000 31513000
0
profile-31512000-31513000.json
41175000 41176000
0
profile-41175000-41176000.json
87057000 87058000
0
profile-87057000-87058000.json
8

0
profile-55619000-55620000.json
7126000 7127000
1
feature_faces
1
53493000 53494000
0
profile-53493000-53494000.json
28806000 28807000
1
feature_faces
1
23499000 23500000
0
profile-23499000-23500000.json
86559000 86560000
0
profile-86559000-86560000.json
74669000 74670000
0
profile-74669000-74670000.json
83526000 83527000
1
feature_faces
1
2060000 2061000
1
feature_faces
1
61601000 61602000
0
profile-61601000-61602000.json
5618000 5619000
0
profile-5618000-5619000.json
37161000 37162000
0
profile-37161000-37162000.json
45504000 45505000
0
profile-45504000-45505000.json
8248000 8249000
1
feature_faces
1
26525000 26526000
1
feature_faces
1
69213000 69214000
0
profile-69213000-69214000.json
42681000 42682000
0
profile-42681000-42682000.json
48116000 48117000
0
profile-48116000-48117000.json
34823000 34824000
0
profile-34823000-34824000.json
47244000 47245000
0
profile-47244000-47245000.json
77542000 77543000
0
profile-77542000-77543000.json
31866000 31867000
1
feature_faces
1
30293000 30

...

feature_faces
1
27546000 27547000
0
profile-27546000-27547000.json
13814000 13815000
0
profile-13814000-13815000.json
53171000 53172000
0
profile-53171000-53172000.json
20823000 20824000
0
profile-20823000-20824000.json
64967000 64968000
0
profile-64967000-64968000.json
92968000 92969000
0
profile-92968000-92969000.json
49967000 49968000
0
profile-49967000-49968000.json
8789000 8790000
0
profile-8789000-8790000.json
86704000 86705000
0
profile-86704000-86705000.json
69015000 69016000
0
profile-69015000-69016000.json
98538000 98539000
0
profile-98538000-98539000.json
64128000 64129000
0
profile-64128000-64129000.json
96409000 96410000
0
profile-96409000-96410000.json
69658000 69659000
0
profile-69658000-69659000.json
91688000 91689000
0
profile-91688000-91689000.json
63133000 63134000
0
profile-63133000-63134000.json
56463000 56464000
0
profile-56463000-56464000.json
68365000 68366000
0
profile-68365000-68366000.json
48027000 48028000
0
profile-48027000-48028000.json
22623000 22624000
0

1
feature_faces
1
44938000 44939000
0
profile-44938000-44939000.json
62576000 62577000
0
profile-62576000-62577000.json
48280000 48281000
0
profile-48280000-48281000.json
48314000 48315000
0
profile-48314000-48315000.json
8792000 8793000
1
feature_faces
1
67338000 67339000
0
profile-67338000-67339000.json
61603000 61604000
0
profile-61603000-61604000.json
24104000 24105000
0
profile-24104000-24105000.json
73577000 73578000
0
profile-73577000-73578000.json
37679000 37680000
0
profile-37679000-37680000.json
56853000 56854000
0
profile-56853000-56854000.json
18189000 18190000
2
feature_faces
1
feature_faces
2
43955000 43956000
0
profile-43955000-43956000.json
77256000 77257000
0
profile-77256000-77257000.json
95457000 95458000
0
profile-95457000-95458000.json
99937000 99938000
0
profile-99937000-99938000.json
52787000 52788000
0
profile-52787000-52788000.json
4307000 4308000
0
profile-4307000-4308000.json
11065000 11066000
0
profile-11065000-11066000.json
50000 51000
0
profile-50000-51000

0
profile-482000-483000.json
32692000 32693000
0
profile-32692000-32693000.json
95341000 95342000
0
profile-95341000-95342000.json
3744000 3745000
0
profile-3744000-3745000.json
72568000 72569000
0
profile-72568000-72569000.json
3577000 3578000
1
feature_faces
1
32589000 32590000
0
profile-32589000-32590000.json
50633000 50634000
0
profile-50633000-50634000.json
15405000 15406000
0
profile-15405000-15406000.json
65343000 65344000
0
profile-65343000-65344000.json
39412000 39413000
0
profile-39412000-39413000.json
30603000 30604000
0
profile-30603000-30604000.json
8289000 8290000
0
profile-8289000-8290000.json
61215000 61216000
0
profile-61215000-61216000.json
29103000 29104000
0
profile-29103000-29104000.json
47486000 47487000
0
profile-47486000-47487000.json
61171000 61172000
0
profile-61171000-61172000.json
15626000 15627000
0
profile-15626000-15627000.json
73155000 73156000
0
profile-73155000-73156000.json
75711000 75712000
0
profile-75711000-75712000.json
91578000 91579000
0
profile

1
feature_faces
1
89216000 89217000
0
profile-89216000-89217000.json
16874000 16875000
0
profile-16874000-16875000.json
36531000 36532000
0
profile-36531000-36532000.json
92972000 92973000
0
profile-92972000-92973000.json
20654000 20655000
0
profile-20654000-20655000.json
59270000 59271000
0
profile-59270000-59271000.json
97194000 97195000
0
profile-97194000-97195000.json
21606000 21607000
0
profile-21606000-21607000.json
87501000 87502000
0
profile-87501000-87502000.json
15419000 15420000
0
profile-15419000-15420000.json
50212000 50213000
0
profile-50212000-50213000.json
28006000 28007000
0
profile-28006000-28007000.json
48823000 48824000
0
profile-48823000-48824000.json
76666000 76667000
0
profile-76666000-76667000.json
39112000 39113000
0
profile-39112000-39113000.json
44887000 44888000
0
profile-44887000-44888000.json
59849000 59850000
0
profile-59849000-59850000.json
74952000 74953000
0
profile-74952000-74953000.json
58917000 58918000
0
profile-58917000-58918000.json
10204000 1020

0
profile-56806000-56807000.json
71880000 71881000
0
profile-71880000-71881000.json
24398000 24399000
0
profile-24398000-24399000.json
76470000 76471000
0
profile-76470000-76471000.json
87089000 87090000
0
profile-87089000-87090000.json
79727000 79728000
0
profile-79727000-79728000.json
89110000 89111000
0
profile-89110000-89111000.json
26904000 26905000
0
profile-26904000-26905000.json
58987000 58988000
0
profile-58987000-58988000.json
67208000 67209000
0
profile-67208000-67209000.json
30061000 30062000
0
profile-30061000-30062000.json
31496000 31497000
0
profile-31496000-31497000.json
59656000 59657000
0
profile-59656000-59657000.json
46704000 46705000
0
profile-46704000-46705000.json
65610000 65611000
0
profile-65610000-65611000.json
18504000 18505000
0
profile-18504000-18505000.json
79160000 79161000
0
profile-79160000-79161000.json
76742000 76743000
0
profile-76742000-76743000.json
2419000 2420000
12
feature_faces
1
93160000 93161000
0
profile-93160000-93161000.json
53473000 53474

In [ ]:
s_embeddings.drop()

In [147]:
s_embeddings.count()

0